In [1]:
! pip install tweepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.2/101.2 kB 701.9 kB/s eta 0:00:00


# Imports

In [2]:
import tweepy
import pandas as pd

# Load Bearer Token

In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
BEAER_TOKEN = user_secrets.get_secret("twitter-bearer-token")

# Client Set Up

In [4]:
client = tweepy.Client(bearer_token=BEAER_TOKEN)

In [5]:
elon = client.get_user(username="elonmusk")

# Get Tweets Demo

In [6]:
tweets = client.get_users_tweets(id='44196397',tweet_fields=["created_at,conversation_id"]).data

In [7]:
parsed_tweets = []
for tweet in tweets:
    parsed_tweets.append([tweet["created_at"],tweet['id'],tweet['text'],tweet['conversation_id']])

In [8]:
df = pd.DataFrame(parsed_tweets)

In [9]:
df.head()

,0,1,2,3
0,2022-10-29 07:08:42+00:00,1586253675583852544,@amasad Good point. \n\nBeing able to select w...,1586078936143867904
1,2022-10-29 06:10:15+00:00,1586238966788857856,@BillyM2k 🎯,1586125659499216896
2,2022-10-29 04:34:43+00:00,1586214927605256192,@WholeMarsBlog One of the best trolls ever 🤣🤣,1586122703013965824
3,2022-10-29 02:19:54+00:00,1586180999364284416,@cosminDZS Something like this makes sense,1586169121913573377
4,2022-10-29 00:21:49+00:00,1586151281571246086,@greg16676935420 Ace reporting by @cnbc 🤣🤣,1586088462788206592


# Get Tweets and Update CSV

In [10]:
parsed_tweets = []
parsed_tweets_only = []
for tweet in tweepy.Paginator(client.get_users_tweets, id='44196397',tweet_fields=["created_at,conversation_id"],
                                    max_results=100).flatten(limit=3200):
    parsed_tweets.append([tweet['id'],tweet["created_at"],tweet['text'],tweet['conversation_id']])
    if tweet['id'] == tweet['conversation_id']:
        parsed_tweets_only.append([tweet['id'],tweet["created_at"],tweet['text']])

In [11]:
tweets_and_replies_df = pd.DataFrame(parsed_tweets)
tweets_df = pd.DataFrame(parsed_tweets_only)

In [12]:
tweets_df.head()

,0,1,2
0,1586149451348910081,2022-10-29 00:14:33+00:00,"To be super clear, we have not yet made any ch..."
1,1586108809772089345,2022-10-28 21:33:03+00:00,Ligma Johnson had it coming 🍆 💦 https://t.co/C...
2,1586104694421659648,2022-10-28 21:16:42+00:00,Comedy is now legal on Twitter
3,1586101267981426689,2022-10-28 21:03:05+00:00,RT @Tesla: Solar + Powerwall ensures you never...
4,1586059953311137792,2022-10-28 18:18:55+00:00,Twitter will be forming a content moderation c...


### Get Last Saved Tweets

In [13]:
old_tweets_and_replies_df = pd.read_csv("../input/elon-musk-tweets-dataset/tweets_and_replies.csv", skiprows = 1, header = None)
old_tweets_df = pd.read_csv("../input/elon-musk-tweets-dataset/tweets.csv",skiprows = 1, header = None)

In [14]:
old_tweets_df.head()

,0,1,2
0,1585841080431321088,2022-10-28 03:49:11+00:00,the bird is freed
1,1585811291851018241,2022-10-28 01:50:49+00:00,Falcon rockets to orbit as seen from LA https:...
2,1585804896841846785,2022-10-28 01:25:24+00:00,RT @SpaceX: Falcon 9’s first stage has landed ...
3,1585804876851789824,2022-10-28 01:25:20+00:00,RT @SpaceX: Liftoff! https://t.co/B0Il6HsYBS
4,1585804859776782336,2022-10-28 01:25:16+00:00,RT @SpaceX: Watch Falcon 9 launch 53 Starlink ...


### Merge Tweets

In [15]:
new_tweets = pd.concat([tweets_df,old_tweets_df]).drop_duplicates()
new_tweets_and_replies = pd.concat([tweets_and_replies_df,old_tweets_and_replies_df]).drop_duplicates()

In [16]:
new_tweets.to_csv("tweets.csv", header=["Id", "Date", "Text"], index=False)
new_tweets_and_replies.to_csv("tweets_and_replies.csv", header=["Id", "Date", "Text", "ConversationId"], index=False)

In [17]:
new_tweets.head()

,0,1,2
0,1586149451348910081,2022-10-29 00:14:33+00:00,"To be super clear, we have not yet made any ch..."
1,1586108809772089345,2022-10-28 21:33:03+00:00,Ligma Johnson had it coming 🍆 💦 https://t.co/C...
2,1586104694421659648,2022-10-28 21:16:42+00:00,Comedy is now legal on Twitter
3,1586101267981426689,2022-10-28 21:03:05+00:00,RT @Tesla: Solar + Powerwall ensures you never...
4,1586059953311137792,2022-10-28 18:18:55+00:00,Twitter will be forming a content moderation c...
